## Voice Conversion (Cross-Lingual)

In [1]:
!conda create -n openvoice python=3.9
!conda activate openvoice
!conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 pytorch-cuda=11.7 -c pytorch -c nvidia
!pip install -r requirements.txt

^C



EnvironmentNameNotFound: Could not find conda environment: openvoice
You can list all discoverable environments with `conda info --envs`.




In [ ]:
import os
import torch
import se_extractor
from api import ToneColorConverter

### Initialization

In [ ]:
ckpt_converter = 'checkpoints/converter'
device = 'cuda:0'
prompt_dir = 'prompts'
skin_dir = 'skins'
output_dir = 'outputs'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(prompt_dir, exist_ok=True)
os.makedirs(skin_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)


### Obtain Tone Color Embedding

The `source_se` is the tone color embedding of the base speaker. 
It is an average for multiple sentences with multiple emotions
of the base speaker. We directly provide the result here but
the readers feel free to extract `source_se` by themselves.

In [ ]:
prompt = f"{prompt_dir}/prompt.wav"
prompt_se, audio_name = se_extractor.get_se(prompt, tone_color_converter, vad=True)

skin = f"{skin_dir}/skin.wav"
target_se, audio_name = se_extractor.get_se(skin, tone_color_converter, vad=True)

### Inference

In [ ]:

save_path = f'{output_dir}/output.wav'

# Run the tone color converter
tone_color_converter.convert(
    audio_src_path=prompt, 
    src_se=prompt_se, 
    tgt_se=target_se, 
    output_path=save_path)